# Required packages

In [1]:
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 7.9/7.9 MB 230.8 kB/s eta 0:00:00
     ------------------------------------ 904.2/904.2 kB 291.9 kB/s eta 0:00:00


In [8]:
from mysql import connector

# Connecting to Mysql

In [22]:
def connectMySQL():
    try :
        conn = connector.Connect(host="localhost",    # your host, usually localhost
                                         user="root",         # your username
                                         port="3306"         # port (3306 default)
                                         )      
        # Creating the cursor
        cur = conn.cursor()
        print("Connected to MySQL")
        return conn,cur
    
    except connector.Error as e:
        print(e)
        return e
        
    

connectMySQL()


Connected to MySQL


(<mysql.connector.connection.MySQLConnection at 0x1b4bd100850>,
 <mysql.connector.cursor.MySQLCursor at 0x1b4bcf446a0>)

# Creating a database

In [26]:
def createDb(nameDB):
    conn,cur=connectMySQL()
    try :
        cur.execute(f"Create database {nameDB}")
        cur.execute("commit")
        # Closing the connection to the default database
        conn.close()
        
    except connector.Error as e :
        print(e)

createDb("DVDrentalDB")
    

Connected to MySQL
1007 (HY000): Can't create database 'dvdrentaldb'; database exists


# Connecting to a database

In [9]:
def connectDB(nameDB):
    try :
        conn = connector.Connect(host="localhost",    # your host, usually localhost
                                 user="root",         # your username
                                 port="3306",         # port (3306 default)
                                 db=nameDB
                                         )      
        # Creating the cursor
        cur = conn.cursor()
        print(f"Connected to {nameDB}")
        return conn,cur
    
    except connector.Error as e:
        print(e)
        return e
    
conn,cur=connectDB("DVDrentalDB")

Connected to DVDrentalDB


# Creating tables

In [1]:
def generateTableCreation():
    tableName=input("Enter the name of the Table : ")
    numberColumns=int(input(f"How many columns are there in {tableName} ? "))
    columns={}
    for i in range(numberColumns):
        nameColumn=input(f"Name of the column N°{i+1} : ")
        while nameColumn in columns.keys():
            nameColumn=input(f"Please choose another name for the column N°{i+1} : ")
        typeColumn=input(f"type of the column N°{i+1} : ")
        columns[nameColumn]=typeColumn
    
    primaryKey=input("Choose the primary key : ")
    while primaryKey not in columns.keys():
        primaryKey=input("Choose a primary key from the existing columns : ")

    
    foreignKeys={}
    numberFk=int(input("how many foreign keys are there ? : "))
    while numberFk > len(columns)-1 and numberFk<0:
        numberFk=int(input("Please enter a valid number of foreign keys : "))
    
    for i in range(numberFk):
        foreignKey=input(f"Enter foreign key N°{i+1} : ")
        while (foreignKey not in columns.keys() ) or  (foreignKey in foreignKeys.keys()):
            foreignKey=input(f"Please enter a valid foreign key : ")
        
        referenceTable=input(f"Enter the reference Table name for the {foreignKey} : ")
        referenceName=input(f"Enter the reference name of {foreignKey} in the {referenceTable} table : ")
        foreignKeys[foreignKey]=[referenceTable,referenceName]
    
    print("\n----------------------\n")
        
    print('"""')
    
    print(f"    CREATE TABLE {tableName} (")
    print(f"    {primaryKey} {columns[primaryKey]} PRIMARY KEY ",end="")
    for k,v in columns.items():
        if k==primaryKey:
            continue
        print(f",\n    {k} {v}",end="")
    
    for key in foreignKeys:
        print(f",\n    FOREIGN KEY ({key}) REFERENCES {foreignKeys[key][0]}({foreignKeys[key][1]})")
    
    print("    );")
    print('"""')
            

In [48]:
generateTableCreation()

Enter the name of the Table : film_category
How many columns are there in film_category ? 3
Name of the column N°1 : film_id
type of the column N°1 : varchar(20)
Name of the column N°2 : category_id
type of the column N°2 : varchar(20)
Name of the column N°3 : last_update
type of the column N°3 : datetime
Choose the primary key : film_id
how many foreign keys are there ? : 2
Enter foreign key N°1 : film_id
Enter the reference Table name for the film_id : film
Enter the reference name of film_id in the film table : film_id
Enter foreign key N°2 : category_id
Enter the reference Table name for the category_id : category
Enter the reference name of category_id in the category table : category_id

----------------------

"""
    CREATE TABLE film_category (
    film_id varchar(20) PRIMARY KEY ,
    category_id varchar(20),
    last_update datetime,
    FOREIGN KEY (film_id) REFERENCES film(film_id)
,
    FOREIGN KEY (category_id) REFERENCES category(category_id)
    );
"""


In [14]:
def createTableWithSql(cur,conn,sqlQuery):
    try:
        cur.execute(sqlQuery)
        conn.commit()
        
        return "Table created!!"
    except connector.Error as e:
        return e



+ **Country table**

In [17]:
createTableWithSql(cur,conn,"""
    CREATE TABLE country (
    country_id varchar(20) PRIMARY KEY ,
    country char(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **City table**

In [23]:
createTableWithSql(cur,conn,"""
    CREATE TABLE city (
    city_id varchar(20) PRIMARY KEY ,
    city char(20),
    country_id varchar(20),
    last_update datetime,
    FOREIGN KEY (country_id) REFERENCES country(country_id)
    );
"""
)

'Table created!!'

+ **Address table**

In [33]:
createTableWithSql(cur,conn,"""
    CREATE TABLE address (
    address_id varchar(20) PRIMARY KEY ,
    address varchar(100),
    address2 varchar(100),
    district varchar(40),
    city_id varchar(20),
    postal_code numeric(6),
    phone numeric(10),
    last_update datetime,
    FOREIGN KEY (city_id) REFERENCES city(city_id)
    );
"""
)

'Table created!!'

+ **Language table**

In [45]:
createTableWithSql(cur,conn,"""
    CREATE TABLE language (
    language_id varchar(5) PRIMARY KEY ,
    name char(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **Film table**

In [47]:
createTableWithSql(cur,conn,"""
    CREATE TABLE film (
    film_id varchar(20) PRIMARY KEY ,
    title varchar(50),
    description char(200),
    release_year year,
    language_id varchar(5),
    rental_duration datetime,
    rental_rate numeric(2),
    length numeric(5),
    replacement_cost numeric(5,2),
    rating numeric(2),
    last_update datetime,
    special_features char(200),
    full_text varchar(100),
    FOREIGN KEY (language_id) REFERENCES language(language_id)
    );
"""
)

'Table created!!'

+ **Category table**

In [37]:
createTableWithSql(cur,conn,"""
    CREATE TABLE category (
    category_id varchar(20) PRIMARY KEY ,
    name varchar(20),
    last_update datetime    );
"""
)

'Table created!!'

+ **Film_Category table**

In [49]:
createTableWithSql(cur,conn,"""
    CREATE TABLE film_category (
    film_id varchar(20) PRIMARY KEY ,
    category_id varchar(20),
    last_update datetime,
    FOREIGN KEY (film_id) REFERENCES film(film_id)
,
    FOREIGN KEY (category_id) REFERENCES category(category_id)
    );
"""
)

'Table created!!'

# Creating tables

In [ ]:
def csvToDb(conn,cur,csv,tableName):
    pass
